# Introduction to Keras for Deep Learning

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [2]:
import numpy as np
from utilities import cifar10
from addutils import css_notebook
css_notebook()

In [3]:
import bokeh.plotting as bk
from bokeh.io import push_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
bk.output_notebook()

Loading BokehJS ...

## 1 Keras

Keras is designed to be modular, minimalist and easily extensible. Francois Chollet, the author of Keras, says:

>The library was developed with a focus on enabling fast experimentation. Being able to go from
idea to result with the least possible delay is key to doing good research.

Keras defines high-level Neural Network modules on top of either TensorFlow or Theano (nowadays has gained less attention thatn TensorFlow). It is possible to compose layer in a modular fashion and even extend the framework with  user defined models.

### 1.1 Installation

Installation is easy. By now you should have the environment *addfor_tutorials* with TensorFlow installed. If either case is not true please refer to the *README.md* to install anaconda and notebook *ml25v04_tensorflow_basic_concepts.ipynb* to install TensorFlow.

Activate your addfor_tutorials environment and from the command-line type:
```
pip install keras
```
Now you can import Keras and check that it is using TensorFlow as its backend.

In [4]:
import keras

Using TensorFlow backend.


In [5]:
print(keras.__version__)

2.1.3


### 1.2 Keras architecture

### 1.2.1 Introduction

Keras uses the backend to perform efficient symbolic computation on Tensors. There are two ways to compose models in Keras: 
 - **Sequential** composition
 - **Functional** composition

The sequential composer build a lists of modules that constituites the architecture of the network, for example a simple feed forward neural network for MNIST can be written as:
```python
model = Sequential()
model.add(Dense(32, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
```

The functional API treat each layer as a function and allows to compose functions into a complex neural network. For example the network defined before can be expressed as somethign like: $$y=relu(f(\sigma(g(x)))$$
The same network can be defined with functional API with:
```python
inputs = Input(shape=(784,))
x = Dense(32)(inputs)
x = Activation("relu")(x)
x = Dense(10)(x)
predictions = Activation("softmax")(x)
model = Model(inputs=inputs, outputs=predictions)
```
Each layer is a function, and since a model is a composition of layers, a model is also a function and can be treated as another layer by calling it on appropriately shaped input tensor. 

Functional API can be used to define any kind of network, but there are some kind of networks that can be defined only using functional API. For example networks with multiple input and outputs or networks that use shared layers. As an example to define a multiple input-output network you can use:

```python
model = Model(inputs=[input1, input2], outputs=[output1, output2])
```

### 1.2.2 Layers Overview

A *Dense model* is a fully connected neural network layer. 

```python
keras.layers.Dense
```

*Convolutional layers* are principally:
```python
keras.layers.convolutional.Conv1D
keras.layers.convolutional.Conv2D
keras.layers.pooling.MaxPooling1D
keras.layers.pooling.MaxPooling2D
```

*Regularization layers:*
```python
keras.layers.core.Dropout
keras.layers.normalization.BatchNormalization
```

*Activation functions*: all principal activation functions are supported.

*Losses*: cross entropy, mean squared error and all popular losses are supported.

*Metrics*: a measure that tells how the model is performing.

*Optimizers*: such as Adam, Adagrad, RMSProp and plain SGD are all supported in module `keras.optimizers`.

### 1.2.3 Training

Beofre training a model, it is necessary to `compile` it. Compile takes three arguments: an optimizer, a loss and a metric.

```python
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```

Once the model is compiled, training is performed by calling the fit method, for example:
```python
# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)
```

### 1.2.4 Additional operations

In Keras is possible to save model architecture to yaml or json format by calling:

```python
# model saving
json_string = model.to_json()
yaml_string = model.to_yaml() 
# model reconstruction
model = model_from_json(json_string)
model = model_from_yaml(yaml_string)
```

Weights are saved in hdf5 format instead, by calling:
```python
model.save('my_model.h5')
```
and restoring with:
```python
model = load_model('my_model.h5')
```

One of the coolest things about Keras is the possiblity of adding callbacks during training. For example the model can decice when to stop based on a EarlyStopping condition or loss history can be saved (and later viewed with TensorBoard) at each iteration. Keras support model checkpointing in a similar way to TensorFlow.

## 2 Simple example

In [6]:
#from keras.datasets import cifar10
#from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import TensorBoard

In [7]:
# image format
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

In [8]:
cifar10.data_path = "example_data/CIFAR-10/"
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [9]:
class_names = cifar10.load_class_names()
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: example_data/CIFAR-10/cifar-10-batches-py/batches.meta
Loading data: example_data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: example_data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: example_data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: example_data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: example_data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: example_data/CIFAR-10/cifar-10-batches-py/test_batch


In [10]:
# constants
BATCH_SIZE = 128
NB_EPOCH = 6
NB_CLASSES = 10
VERBOSE = 0
VALIDATION_SPLIT = 0.2

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', 
                 input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [13]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

In [14]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracy = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('acc'))

In [15]:
callbacks = []
callbacks.append(TensorBoard(log_dir='temp/keras/logs'))

In [16]:
history_new = LossHistory()
callbacks.append(history_new)

In [17]:
hist = model.fit(images_train, labels_train, batch_size=BATCH_SIZE,
                 epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, 
                 verbose=VERBOSE, callbacks=callbacks)

In [18]:
score = model.evaluate(images_test, labels_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print('Test accuracy: {}'.format(score[1])) 

Test accuracy: 0.6635


In [19]:
print(hist.history)

{'val_loss': [1.2854390144348145, 1.1528461042404174, 1.0491638912200927, 0.9538292743682861, 1.049972660446167, 0.9514888729095459], 'val_acc': [0.5575, 0.5949, 0.6337, 0.6694, 0.6411, 0.6674], 'loss': [1.6673770608901977, 1.2886534355163575, 1.1307520782470704, 1.0347720762252808, 0.9603295879364013, 0.9029679985046387], 'acc': [0.40125, 0.544675, 0.60585, 0.636675, 0.665725, 0.689625]}


In [20]:
fig = bk.figure(plot_width=500, plot_height=300, title=None)
fig.line(np.array(range(len(history_new.losses))), np.array(history_new.losses))
fig.line(np.array(range(len(history_new.accuracy))), np.array(history_new.accuracy), 
         color='red')
bk.show(fig)

---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.